In [5]:
#Current imports
import pandas as pd

from sklearn.preprocessing import LabelEncoder, StandardScaler

##Tensorflow
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense


In [7]:
##Read in CSV
df = pd.read_csv('IMDB_Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
y=df['sentiment']
##turn string negative to positive into binary classification




In [ ]:
##Preprocessing libraries
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
##Preprocessing code
##remove stop words?
##lowercase or standardize the format or each word?
##grab movie reviews
##parse it


In [ ]:
def doc2vec_model():
    ##Sequential right???
    model = Sequential()

    #Embedding layer
    #GlobalAveragePooling1D layer
    #Flattening

    #Dense layers
    #Optimization such as dropout??


    #binary or multi-class compilation
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
##Train actual model


model = doc2vec_model()
##model.fit(, epochs=20, batch_size=10, verbose=1)